In [2]:
import pandas as pd
import numpy as np
import pprint as pp

In [3]:
from google.colab import files
uploaded = files.upload()
df= pd.read_csv('creditcard.csv')

Saving creditcard.csv to creditcard.csv


In [7]:
from collections import Counter
def entropy_list(a_list):
    cnt = Counter(x for x in a_list)
    num_instance = len(a_list)*1.0
    probs = [x/num_instance for x in cnt.values()]
    return entropy(probs)
import math
def entropy(probs):
    return sum([-prob*math.log(prob,2) for prob in probs])
def entropy(probs):
     return sum([-prob*math.log(prob,2) for prob in probs])
def info_gain(df,split,target,trace=0):
     df_split = df.groupby(split)
     nobs = len(df.index)*1.0
     df_agg_ent = df_split.agg({ target:[entropy_list, lambda x: len(x)/nobs] })
     df_agg_ent.columns = ['Entropy','PropObserved']
     new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent["PropObserved"])
     old_entropy = entropy_list(df[target])
     return old_entropy - new_entropy

In [8]:
def id3(df,target,attribute_name,default_class = None):
    cnt = Counter(x for x in df[target])
    if len(cnt)==1:
        return next(iter(cnt))
    elif df.empty or (not attribute_name):
        return default_class
    else:
        default_class = max(cnt.keys())
        gains = [info_gain(df,attr,target) for attr in attribute_name]
        index_max = gains.index(max(gains))
        best_attr = attribute_name[index_max]
        tree = { best_attr:{ } }
        remaining_attr = [x for x in attribute_name if x!=best_attr]
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,target,remaining_attr,default_class)
            tree[best_attr][attr_val] = subtree
        return tree
def classify(instance,tree,default = None):
    attribute = next(iter(tree))
    if instance[attribute] in tree[attribute].keys():
        result = tree[attribute][instance[attribute]]
        if isinstance(result,dict):
            return classify(instance,result)
        else:
            return result
    else:
        return default

In [11]:
attribute_names = list(df.columns)
attribute_names.remove('Class') #Remove the class attribute
tree = id3(df,'Class',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")
pp.pprint(tree)
training_data = df.iloc[1:-4] # all but last thousand instances
test_data = df.iloc[-4:] # just the last thousand
train_tree = id3(training_data, 'Class', attribute_names)
print("\n\nThe Resultant Decision train_tree is :\n")
pp.pprint(train_tree)
test_data['predicted2'] = test_data.apply(classify,axis=1,args=(train_tree,0) )
print ('\n\n Training the model for a few samples, and again predicting outcome for remaining attribute')
print('The Accuracy for new trained data is : ' + str( sum(test_data['Class']==(test_data['predicted2']) ) / (1.0*len(test_data.index)) ))


Streaming output truncated to the last 5000 lines.
        1.35866243942799: 0.0,
        1.35866395053724: 0.0,
        1.35868203304765: 0.0,
        1.35872022098389: 0.0,
        1.35872239192843: 0.0,
        1.3587235206226: 0.0,
        1.3587334355695: 0.0,
        1.358735495324: 0.0,
        1.35874179470754: 0.0,
        1.35878915008512: 0.0,
        1.35879403470005: 0.0,
        1.35881382732347: 0.0,
        1.35881683495085: 0.0,
        1.35885336709095: 0.0,
        1.35888318975604: 0.0,
        1.3589257469246: 0.0,
        1.35893567440361: 0.0,
        1.35894427761197: 0.0,
        1.35895081835723: 0.0,
        1.35895234282839: 0.0,
        1.35899318931744: 0.0,
        1.35900245318505: 0.0,
        1.35900421241203: 0.0,
        1.3590075489346: 0.0,
        1.35902691890482: 0.0,
        1.35904963749321: 0.0,
        1.35905989370693: 0.0,
        1.35908429039245: 0.0,
        1.35910407081913: 0.0,
        1.35911684671761: 0.0,
        1.35916169133616:

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
